In [ ]:
# Class to define entries in the HashTable
class TableEntry:
    def __init__(self, key, value):
        self._key = key
        self._value = value
        self._in_table = True
        
    def get_key(self):
        return self._key
    
    def get_value(self):
        return self._value
    
    def set_value(self, value):
        self._value = value
    
    def is_in_table(self):
        return self._in_table
    
    def set_to_removed(self):
        self._key = None
        self._value = None
        self._in_table = False
    
    def is_removed(self):
        return (self._key is None) and (self._value is None) and not self._in_table

In [ ]:
# Hash Table implementation
class HashTableWithCount:
    def __init__(self, initial_capacity=0): # choose an initial capacity
        pass
    
    def get_next_prime(self, n):
        pass
    def is_prime(self, n):
        pass
    def check_size(self, size):
        pass
    def get_hash_index(self, key):
        pass
    def is_hash_table_too_full(self):
        pass
    def enlarge_hash_table(self):
        pass
    def reset_probe_count(self):
        pass
    def get_probes(self):
        pass

In [ ]:
# Inherits HashTableWithCount using linear probing
class LinearProbingWithCount:
    def probe(self, index, key):
        # code here
         return index
    
        
    
    #locate is used to return an index of a key that it is given. 
    # So it receives an index to start searching at and a key to 
    # search for and then returns the index of where the key is.
    def locate(self, index, key):   # this function is optional, see below
        pass
    def add(self, key, value):
        pass
    def remove(self, key):
        pass
    def get_value(self, key):
        pass
    def contains(self, key):
        pass
    def is_empty(self):
        pass
    def get_size(self):
        pass
    def clear(self):
        pass

In [ ]:
# Inherits HashTableWithCount using double hashing
class DoubleHashingWithCount:
    def probe(self, index, key):
        return index
    
    #locate is used to return an index of a key that it is given. 
    # So it receives an index to start searching at and a key to 
    # search for and then returns the index of where the key is.
    def locate(self, index, key):  # this function is optional, see below
        pass
    def add(self, key, value):
        pass
    def remove(self, key):
        pass
    def get_value(self, key):
        pass
    def contains(self, key):
        pass
    def is_empty(self):
        pass
    def get_size(self):
        pass
    def clear(self):
        pass

In [ ]:
# Where you run the experiment.
class GetStatistics:
    def main():
        pass
    def choose_100_names_and_add(linear_ht, double_ht, names):
        pass
    def get_1000_names(names):
        pass
    def get_10000_names(names):
        pass
    def search_for_100_names(linear_ht, double_ht, names):
        pass